# Conservative Zonal Averaging (Latitude Bands)

This notebook introduces zonal averaging and conservation in a step-by-step, visual way.

Key ideas:
- A zonal mean summarizes values along latitude.
- Non-conservative: averages along individual lines of latitude.
- Conservative (banded): averages over finite latitude bands, weighting by area. This conserves integral quantities.


In [ ]:
# Ensure we import the local uxarray package from the repository root
import os
import sys

sys.path.insert(0, os.path.abspath(os.path.join("..", "..")))

import cartopy.crs as ccrs
import geoviews as gv
import geoviews.feature as gf
import holoviews as hv
import numpy as np

hv.extension("bokeh")

import uxarray as ux

## Load a sample dataset
We use a standard unstructured grid test case and a scalar field `psi`. The map shows the raw field.

In [ ]:
projection = ccrs.Robinson()
uxds = ux.open_dataset(
    "../../test/meshfiles/ugrid/outCSne30/outCSne30.ug",
    "../../test/meshfiles/ugrid/outCSne30/outCSne30_vortex.nc",
)
uxds["psi"].plot(
    cmap="inferno",
    periodic_elements="split",
    projection=projection,
    title="Global Field",
)

## Zonal mean: concept
- A latitude circle is the set of all points at a fixed latitude.
- The non-conservative zonal mean computes an average along such a circle.
- The conservative (banded) zonal mean instead averages across a latitude interval (band), weighting every face by its area inside the band.

The banded approach preserves integrals: the sum over bands of (band mean × band area) matches the global area integral (up to numerical quadrature error).


In [ ]:
# Non-conservative: lat grid every 5 degrees
z_nc = uxds["psi"].zonal_mean(lat=(-90, 90, 5))
z_nc

In [ ]:
# Conservative banded: same tuple interpreted as band edges
z_c = uxds["psi"].zonal_mean(lat=(-90, 90, 5), mode="conservative")
edges = z_c.attrs.get("lat_band_edges")
centers = 0.5 * (edges[:-1] + edges[1:])
z_c, edges

## Visualize latitude bands on the map
Shaded rectangles show example bands that the conservative scheme aggregates over.

In [ ]:
# Helper to draw a band as a filled rectangle (easy shaded overlay)
def band_rect(min_lat, max_lat, lon_min=-180, lon_max=180):
    return gv.Rectangles([(lon_min, min_lat, lon_max, max_lat)], crs=ccrs.PlateCarree())


# Build a subset of bands for visualization (e.g., every 15 degrees)
band_edges_vis = np.arange(-90, 90 + 15, 15)
bands_layer = None
for a, b in zip(band_edges_vis[:-1], band_edges_vis[1:]):
    rect = band_rect(a, b).opts(fill_alpha=0.18, line_alpha=0.0, color="steelblue")
    bands_layer = rect if bands_layer is None else bands_layer * rect

(
    uxds["psi"].plot(
        cmap="inferno",
        periodic_elements="split",
        projection=projection,
        title="Latitude Bands (shaded)",
    )
    * bands_layer
    * gf.coastline(projection=projection)
    * gf.grid(projection=projection)
)

## Compare the two schemes (line plot)
We overlay the non-conservative curve with the conservative (banded) curve. A highlighted x-span marks one band; we annotate the conservative band mean at its center.

In [ ]:
# Choose a band to highlight (e.g., centered near the equator)
highlight_idx = np.argmin(np.abs(centers - 0.0))
x0, x1 = edges[highlight_idx], edges[highlight_idx + 1]
xmid = centers[highlight_idx]
ymid = (
    float(z_c.sel(latitudes=xmid))
    if "latitudes" in z_c.coords
    else float(z_c.values[..., highlight_idx])
)

line_nc = z_nc.plot.line(label="non-conservative")
line_c = z_c.plot.line(label="conservative (bands)", line_dash="dashed")
span = hv.VSpan(x0, x1).opts(alpha=0.12, color="steelblue")
pt = hv.Scatter([(xmid, ymid)]).opts(color="crimson", size=8)
txt = hv.Text(xmid, ymid, f"mean={ymid:.3f}").opts(text_color="crimson")

(span * line_nc * line_c * pt * txt).opts(
    title="Zonal Mean: Non-conservative vs Conservative (5°)",
    xlabel="latitudes (deg)",
    ylabel=uxds["psi"].name or "value",
    xticks=np.arange(-90, 91, 30),
    xlim=(-90, 90),
)

## Conservation diagnostics
We verify that the band integration is both geometrically and physically consistent.

1) Geometry: Sum of per-face overlap areas within a band should match the analytic spherical band area: 2π · (sin φ₂ − sin φ₁).
2) Conservation: The sum over bands of (band_mean × band_area) should match the global area integral of the field over the same latitude range.


In [ ]:
# Compute analytic band areas on unit sphere
phi = np.deg2rad(edges)
band_areas_analytic = 2.0 * np.pi * (np.sin(phi[1:]) - np.sin(phi[:-1]))

# Geometric area via face areas
face_areas = uxds.uxgrid.face_areas.values
A_faces_total = face_areas.sum()
A_analytic_total = band_areas_analytic.sum()

print("Analytic sphere area fraction (lat -90..90):", A_analytic_total)
print("Sum of face areas (mesh):                 ", A_faces_total)

In [ ]:
# Physical conservation: compare band-summed integral vs direct face integral
psi = uxds["psi"]

# Direct face-area integral
I_faces = np.einsum("i,i->", uxds.uxgrid.face_areas.values, psi.values)

# Band integral from conservative means: sum_b( mean_b * A_b(analytic) )
z_c_vals = z_c.values  # band means
I_bands = float((z_c_vals * band_areas_analytic).sum())

print("Direct face integral: ", I_faces)
print("Banded integral:      ", I_bands)
print("Relative diff:        ", abs(I_faces - I_bands) / max(1e-16, abs(I_faces)))

### Sanity checks
- Constant field: band means are all 1, banded integral equals total area.
- Smooth fields: reducing band width (e.g., from 15° to 5°) should bring conservative and non-conservative curves closer.


## Choosing custom bands
You can provide explicit band edges via the `bands=` argument. For example, coarser 15° bands:

In [ ]:
edges15 = np.arange(-90, 90 + 15, 15)
z_c15 = uxds["psi"].zonal_mean(mode="conservative", bands=edges15)
z_c15.plot.line(color="firebrick").opts(
    title="Conservative Zonal Mean (15° bands)",
    xticks=np.arange(-90, 91, 30),
    xlim=(-90, 90),
)